# Логистическая регрессия для задач анализа текстов
Для набора твитов вам будет необходимо определить обучить модель логистической регрессии для определения тональности твитов.

## Import functions and data

In [1]:
!wget https://www.dropbox.com/s/nxdok8u3d24re5n/utils.py

--2021-12-27 17:47:30--  https://www.dropbox.com/s/nxdok8u3d24re5n/utils.py
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nxdok8u3d24re5n/utils.py [following]
--2021-12-27 17:47:31--  https://www.dropbox.com/s/raw/nxdok8u3d24re5n/utils.py
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca8cbda63621bdcfdf048278c16.dl.dropboxusercontent.com/cd/0/inline/Bcr2AOk_OKVR7fjuAhxFoDh7cG0R1NFDhiBg4R-MeVBhfbbBjXPiRd14Jm-K-WMinuJFFxDcHXk3axJWXMgYmaCZldpyB2StzaeVDJKvjM9O_ANzrvKMb5DXDSYvfOnrh27lMrLzPgD8gmCx4Vsis8we/file# [following]
--2021-12-27 17:47:31--  https://uca8cbda63621bdcfdf048278c16.dl.dropboxusercontent.com/cd/0/inline/Bcr2AOk_OKVR7fjuAhxFoDh7cG0R1NFDhiBg4R-MeVBhfbbBjXPiRd14Jm-K-WMinuJFFxDcHXk3axJWXMgYmaCZldpyB2StzaeVDJ

In [2]:
!wget https://www.dropbox.com/s/0cunnvacrz5kur3/w1_unittest.py

--2021-12-27 17:47:32--  https://www.dropbox.com/s/0cunnvacrz5kur3/w1_unittest.py
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0cunnvacrz5kur3/w1_unittest.py [following]
--2021-12-27 17:47:32--  https://www.dropbox.com/s/raw/0cunnvacrz5kur3/w1_unittest.py
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucefd68990cceec17c49d2822dd9.dl.dropboxusercontent.com/cd/0/inline/BcrqOT2s1T2kcvKB4LnXEPqEvmONK4P9RQ4gFcSsIA6CPFBCBusV86C-fyrDZcXiMU4UngfSLnanwMz4K-YHByV_1SR-5EfZQadff2DSt3J6NgIlpyITcxCa1B6kwsqw8Hf2d3-gNUoMmpIDZSSIvgMV/file# [following]
--2021-12-27 17:47:32--  https://ucefd68990cceec17c49d2822dd9.dl.dropboxusercontent.com/cd/0/inline/BcrqOT2s1T2kcvKB4LnXEPqEvmONK4P9RQ4gFcSsIA6CPFBCBusV86C-fyrDZcXiMU4UngfSLnanwMz4K-YH

In [3]:
import nltk
from os import getcwd
# Модуль для тестирования
import w1_unittest


nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Imported functions

[Документация для датасета](http://www.nltk.org/howto/twitter.html).

#### В модуле utils.py реализованы некоторые полезные функции для предобработки данных и выполнения задания:
* process_tweet: производит предобработку твитов: удаляются стоп-слова, производится токенизация и стемминг.
* build_freqs: производится рассчет количества присутствия определенного слова в корпусе для твитов, принадлежащих классу '1' или '0', затем строится словарь частот 'freqs', где ключом является пара (word,label), а значением - соответствующая частота.

In [4]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [5]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs

### Подготовка данных
* `twitter_samples` датасет, содержащий 5000 поизитивных и 5000 негативных твитов.  

In [6]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

* Train test split: 20% для тестового набора и 80% - для обучения.


In [7]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

* Создаем numpy-массив для позитивныз и негативных меток классов.

In [8]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [9]:
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


* Посмотрите в модуле utils.py как работает функция создания частотного словаря

```Python
    for y,tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
```

In [10]:
# строим частотный словарь
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11436


#### Ожидаемый вывод
```
type(freqs) = <class 'dict'>
len(freqs) = 11436
```

### Обработка твитов
С помощью функции 'process_tweet' разделите твиты на список слов, удалите стоп-слова и проведите стемминг.

In [11]:
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


#### Ожидаемый вывод
```
This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
 
This is an example of the processes version: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']
```

# Часть 1: Логистическая регрессия


### Часть 1.1: Сигмойда
 
* Функция сигмойды определяется следующим образом: 

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

Она отображает входное значение 'z' в число, лежащее на отрезке от 0 до 1, и может рассматриваться как вероятность. 

#### Оеализуйте вычисление сигмойды
* Функция должна работать как со скалярным z так и с вектором

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Подсказки</b></font>
</summary>
<p>
<ul>
    <li><a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html" > numpy.exp </a> </li>

</ul>
</p>



In [12]:
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
    ### START CODE HERE ###
    # Вычисление сигмойды z
    h = 1/(1+np.exp(-z))
    ### END CODE HERE ###
    
    return h

In [13]:
# Testing your function 
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

SUCCESS!
CORRECT!


In [14]:
# Test your function
w1_unittest.test_sigmoid(sigmoid)

 All tests passed


### Логистическая регрессия: Регрессия и сигмойда

Логистическая регрессия - это, по сути, применение сигмойды к выводу линейной регрессии.

Регрессия:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
$\theta$ - эта веса модели "weights". 

Логистическая регрессия
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
'z' называются логитами ('logits').

### Часть 1.2 Функция потерь и градиент

Функция потерь, используемая при обучении логистической регрессии - это log loss, усредненный по всем обучающим примерам:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ - количество обучающих примеров
* $y^{(i)}$ - истинный класс 'i'-го объекта обучающей выборки.
* $h(z^{(i)})$ - предсказание модели для 'i'-го объекта обучающей выборки.

Значение функции потерь для одного объекта определяется следующим образом:
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* Все значения $h$ лежат на отрезке от 0 до 1, так что logs принимают отрицательное значение. Поэтому функция потерь умножается на -1.
* Заметьте, что если модель дает предсказание 1 ($h(z(\theta)) = 1$) и истинный класс $y_{true}$ также равен 1, то loss на обучении равен 0. 
* Аналогично, если модель предсказывает 0 ($h(z(\theta)) = 0$) и $y_{true}=0$, то loss = 0. 
* Однако, если предсказание модели близко к 1 ($h(z(\theta)) = 0.9999$), а $y_{true}=0$, то второе слагаемое в формуле loss станет большим отрицательным числом, после умножения на -1, значение станет большим положительным числом. $-1 \times (1 - 0) \times log(1 - 0.9999) \approx 9.2$. При этом, чем ближе значение модели будет к 1, тем больше будет становится loss.

In [15]:
-1 * (1 - 0) * np.log(1 - 0.9999) # loss is about 9.2

9.210340371976294

* Аналогично, если предсказание модели близко к 0 ($h(z) = 0.0001$), но $y_{true} = 1$, то первое слагаемое в формуле будет большим числом: $-1 \times log(0.0001) \approx 9.2$. Чем ближе предсказание к 0, тем больше становится loss.

In [16]:
-1 * np.log(0.0001) # loss is about 9.2

9.210340371976182

#### Обновление весов

Для обновления весов $\theta$, необходимо реализовать градиентный спуск для улучшения качества прогнозов модели и уменьшения loss.  
Градиент функции потерь $J$, вычисленный относительного одного из весов $\theta_j$:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x^{(i)}_j \tag{5}$$
* 'i' индекс по всем 'm' обучающим примерам.
* 'j' индекс веса $\theta_j$, так что $x^{(i)}_j$ - это признак, при котором стоит вес $\theta_j$

* Для обновления веса $\theta_j$, мы вычитаем из коэффициента, полученного на предыдущем шаге градиент с коэффициентом $\alpha$:
$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta) $$
* Коэффициент $\alpha$ - это шаг, который контролирует, насколько сильно мы будем двигаться в сторону антиградиента.


## Задание: Реаизуйте функцию градиентного спуска
* Количество итераций 'num_iters" - это количсетво проходов по всему обучающему набору.
* На каждой итерации, вам необходимо вычислить функцию потерь для всех обучающих примеров ('m'), и по всем признакам.
* Вместо обновления одного веса $\theta_i$ за раз, будем обновлять весь вектор сразу:  
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$
* Размерность $\mathbf{\theta}$ - (n+1, 1), где 'n' - число признаков, $\theta_0$ - это вес, который стоит при фиктивном константном признаке 1 (bias) ($\mathbf{x_0}$ = 1).
* 'logits', 'z', вычисляются умножением матрицы объекты-признаки 'x' на вектор весов 'theta'.  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ имеет размерность (m, n+1) 
    * $\mathbf{\theta}$: имеет размерность (n+1, 1)
    * $\mathbf{z}$: имеет размерность (m, 1)
* Предсказание 'h' вычисляется применением сигмойды 'z': $h(z) = sigmoid(z)$ и имеет размерность (m,1).
* Функция потерь $J$ вычисляется как скалярное произведение 'y' и 'log(h)'.  Так как оба 'y' и 'h' имеют размерность (m,1), необходимо транспонировать вектор слева, так что умножение строки на колонку будет представлять скалярное произведение.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* Обновление весов также будет происходит в векторной форме.  Поскольку размерность $\mathbf{x}$ - (m, n+1), а размерность $\mathbf{h}$ и $\mathbf{y}$ - (m, 1), нужно транспонировать $\mathbf{x}$ и поставить его слева при умножении матриц, тогда мы получим нужный ответ размерности (n+1, 1):
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Подсказки</b></font>
</summary>
<p>
<ul>
    <li>используйте numpy.dot для умножения матриц.</li>
    <li>Убедитесь, что -1/m является вещественным значением (не целым), можно написать `float(1)` или `1.` </li>
</ul>
</p>



In [17]:
# UNQ_C2 GRADED FUNCTION: gradientDescent
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE ###
    # 'm' - количество строк в матрице x
    m = len(x)
  
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x,theta)
        
        # get the sigmoid of z
        h = sigmoid(z)
        
        # calculate the cost function
        J = (-1/m)*(np.dot(y.T,np.log(h)) + np.dot((1-y).T,np.log(1-h)))
        
        # update the weights theta
        theta = theta - (alpha/m)*np.dot(x.T, h-y)
        
    J = float(J)
    return J, theta

In [18]:
# Проверка работоспособности функции
np.random.seed(1)

# создаем синтетический набор данных
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)

tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Градиентный спуск
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]



#### Ожидаемый результат
```
The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]
```

In [19]:
# Протестируйте функцию
w1_unittest.test_gradientDescent(gradientDescent)

 All tests passed


## Часть 2: Выделение признаков

* Получив список твитов, извлеките признаки и сохраните их в матрице. 

 * Первый признак - это количество положительных слов в твите.
 * Второй признак - это количество отрицательных слов в твите. 
* Затем обучите свой классификатор логистической регрессии по этим признаковым описаниям твитов.
* Протестируйте классификатор на валидационном анаборе. 

### Задание: Реализуйте функцию extract_features. 
* Эта функция принимает один твит.
* Обработайте твит с помощью импортированной функции process_tweet и сохраните список слов твита.
* Повторите цикл по каждому слову в списке обработанных слов
 * Для каждого слова проверьте в частотном словаре 'freqs' количество случаев, когда это слово имеет положительную метку "1". (Проверьте наличие ключа (слово, 1.0)
 * Сделайте то же самое для подсчета связи слова с отрицательной меткой "0". (Проверьте наличие ключа (слово, 0.0))



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Подсказки</b></font>
</summary>
<p>
<ul>
    <li>Убедитесь, что вы правильно обрабатываете случаи, когда ключ (word, label) не найден в словаре. </li>
</ul>
</p>


In [20]:
def extract_features(tweet, freqs, process_tweet=process_tweet):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet - предобработка твитов
    word_l = process_tweet(tweet)
    
    # 3 elements в форме вектора 1 x 3 
    x = np.zeros((1, 3)) 
    
    #bias - 1
    x[0,0] = 1 
    
    ### START CODE HERE ###
    
    # Проходим по всем словам в списке
    for word in word_l:
        
        # увеличиваем счетчик при появлении метки 1
        x[0,1] += freqs.get((word,1),0)
        
        # увеличиваем другой счетчик при появлении метки 0
        x[0,2] += freqs.get((word,0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [21]:
# Проверка функции
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.000e+00 3.133e+03 6.100e+01]]


#### Ожидаемый вывод
```
[[1.000e+00 3.133e+03 6.100e+01]]
```

In [22]:
# Проверка функции
# Если слова не было в словаре
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


#### Ожидаемый вывод
```
[[1. 0. 0.]]
```

In [23]:
# Тестирование функции
w1_unittest.test_extract_features(extract_features, freqs)

 All tests passed


## Часть 3: Обучение модели

Для обучения модели:
* Соберите все признаки для обучающих примеров в матрицу X. 
* Вызовите функцию `gradientDescent`, которую вы реализовали выше.

Эта часть реализована, прочитайет внимательно код и убедитесь, что вы все поняли.

In [24]:
# собираем признаки в матрицу 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# метки для обучающих примеров
Y = train_y

# Применяем градиентный спуск
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.22522315.
The resulting vector of weights is [6e-08, 0.00053818, -0.0005583]


**Ожидаемый результат**: 

```
The cost after training is 0.22522315.
The resulting vector of weights is [6e-08, 0.00053818, -0.0005583]
```

# Часть 4: Тестирование логистической регрессии

#### Задание: реализуйте функцию `predict_tweet`
предскажите, обладает ли твит положительной или отрицательной окраской.

* Проиизведите предобработку новых твитов и извлеките для них признаки.
* Примените обученную модель к признаковым описаниям новых твитов.
* Примените сигмойду к логитам для предсказания класса (значение от 0 до 1).

$$y_{pred} = sigmoid(\mathbf{x} \cdot \theta)$$

In [25]:
# UNQ_C4 GRADED FUNCTION: predict_tweet
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE ###
    
    # выделение признаков
    x = extract_features(tweet, freqs)
    
    # предсказание
    y_pred = sigmoid(np.dot(x, theta))
    
    ### END CODE HERE ###
    
    return y_pred

In [26]:
# Тестирование функции
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))    

I am happy -> 0.519275
I am bad -> 0.494347
this movie should have been great. -> 0.515979
great -> 0.516065
great great -> 0.532096
great great great -> 0.548062
great great great great -> 0.563929


**Ожидаемый результат**: 
```
I am happy -> 0.519275
I am bad -> 0.494347
this movie should have been great. -> 0.515979
great -> 0.516065
great great -> 0.532096
great great great -> 0.548062
great great great great -> 0.563929
```

In [27]:
# Проверьте на собственных твитах
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

array([[0.83110307]])

In [28]:
# Протестируйте функцию
w1_unittest.test_predict_tweet(predict_tweet, freqs, theta)

 All tests passed


## Оцените качество работы модели на тестовом наборе
После обучения вашей модели с использованием приведенного выше обучающего набора проверьте, как ваша модель может работать с реальными, протестировав ее на тестовом наборе.

#### Задание: Реализовать `test_logistic_regression` 
* Учитывая тестовые данные и веса вашей обученной модели, рассчитайте точность вашей модели логистической регрессии. 
* Используйте функцию "predict_tweet", чтобы делать прогнозы по каждому твиту в тестовом наборе.
* Если прогноз > 0,5, установите классификацию модели 'y_hat' равной 1, в противном случае установите классификацию модели 'y_hat' равной 0.
* Прогноз является точным, когда y_hat равно test_y. Суммируйте все случаи, когда они равны, и разделите на m.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Подсказки</b></font>
</summary>
<p>
<ul>
    <li>Используйте np.asarray() для перевода list в numpy массив</li>
    <li>Используйте numpy.squeeze() для того чтобы убрать ненужную размерность ((m,1) -> (m,)) [[]] -> [] </li>
</ul>
</p>

In [29]:
def test_logistic_regression(test_x, test_y, freqs, theta, predict_tweet=predict_tweet):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### START CODE HERE ###
    
    # Список предсказаний
    y_hat = []
    
    for tweet in test_x:
        # метка для предсказанного твита
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # добавляем 1.0 к списку
            y_hat.append(1.0)
        else:
            # добавляем 0 к списку
            y_hat.append(0.0)

    # Сейчас y_hat - это list, f test_y - массив (m,1) (array)
    # переведите обе переменные к типу one-dimensional arrays, чтобы использовать оператор сравнения '=='
    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)

    ### END CODE HERE ###
    
    return accuracy

In [30]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950


#### Ожидаемый результат: 
```0.9950```  
Pretty good!

In [31]:
# Протестируйте функцию
w1_unittest.unittest_test_logistic_regression(test_logistic_regression, freqs, theta)

 All tests passed


# Part 5: Анализ ошибок

В этой части вы увидите несколько твитов, которые ваша модель неправильно классифицировала. Как вы думаете, почему произошла неправильная классификация? В частности, какие твиты неправильно классифицирует ваша модель?

In [32]:
# Анализ ошибок
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
THE TWEET IS: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
THE PROCESSED TWEET IS: ['sure', 'would', 'good', 'thing', '4', 'bottom', 'dare', '2', 'say', '2', 'miss', 'b', 'im', 'gonna', 'stubborn', 'mouth', 'soap', 'nothavingit', ':p']
1	0.48901497	b'sure would good thing 4 bottom dare 2 say 2 miss b im gonna stubborn mouth soap nothavingit :p'
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
THE PROCESSED TWEET IS: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48418949	b"i'm play brain dot braindot"
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots http://t.co/aOKldo3GMj http://t.co/xWCM9qyRG5
THE PROCESSED TWEET IS: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48418949	b"i'm play brain dot braindot"
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots http://t.co/R2JBO8iNww http://t.co/ow5BBwdEMY
THE PROCESSED TWEET IS: ["i'm", 'play', 

# Part 6: Предсказание для собственных твитов

In [33]:
# Придумайте и предскажите тональность собственных, интересных твитов
my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['ridicul', 'bright', 'movi', 'plot', 'terribl', 'sad', 'end']
[[0.48125423]]
Negative sentiment
